In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import *
from mycelia.shared.chain import * 
from mycelia.miner.train import * 
from mycelia.miner.model_io import *
from mycelia.shared.checkpoint import *
import bittensor as bt 
from mycelia.shared.app_logging import *

configure_logging()
logger = structlog.get_logger(__name__)

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subtensor = bt.Subtensor('test')
path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
miner_config = MinerConfig.from_path(path)

path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
vali_config = ValidatorConfig.from_path(path)

rank = 2
wallet, subtensor = setup_chain_worker(miner_config)
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(miner_config)
expert_manager = ExpertManager(miner_config)

2025-12-23 00:43:18 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-23 00:43:18 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


In [11]:
# --- Get model from chain ---
def scan_chain_for_new_model(
    current_model_meta: ModelMeta | None,
    config: WorkerConfig,
    subtensor: bittensor.Subtensor,
) -> tuple[bool, list[dict]]:
    """
    Returns:
        should_download: True if a newer model (by version) is available and a majority
                         agree on the model_hash among those newer entries.
        download_meta:   list of dicts with fields: uid, ip, port, model_hash, model_version
                         filtered to entries that (a) are newer and (b) match the majority hash.
    """
    commits: tuple[WorkerChainCommit, bittensor.Neuron] = get_chain_commits(config, subtensor)

    max_model_meta = ModelMeta(
        global_ver=max(
            (c.global_ver for c, n in commits if c is not None and getattr(c, "global_ver", None) is not None),
            default=0,
        )
    )

    if current_model_meta is not None:
        logger.info(
            "Scan chain: Max model version on chain",
            max_model_version_on_chain=max_model_meta,
        )
        logger.info(
            "Scan chain: Local model version",
            current_model_version=current_model_meta,
        )
        max_model_meta = max(max_model_meta, current_model_meta)
    
    logger.info(
        "Scan chain: Max model meta",
        max_model_meta = max_model_meta,
    )
    # 0) Download only from validator
    # commits = [(c, n) for c, n in commits if n.validator_permit]
    commits = [(c, n) for c, n in commits if getattr(c, "miner_seed", False)]
    commits = [(c, n) for c, n in commits if getattr(c, "model_hash", False)]

    for c in commits:

        logger.info('commits A', c)
    # 1) collect candidates that are newer than the current version
    most_updated_commits = []  # type: ignore
    for c, n in commits:
        raw_ver = getattr(c, "global_ver", None)
        global_ver = raw_ver if isinstance(raw_ver, int) else 0
        logger.info(c, raw_ver, global_ver)
        if ModelMeta(global_ver=global_ver) >= max_model_meta:
            most_updated_commits.append((c, n))

    logger.info('commits B', most_updated_commits)
    if len(most_updated_commits) == 0:
        return False, []

    # 2) majority filter by model_hash among the newer candidates
    hash_counts = Counter([c.model_hash for c, n in most_updated_commits if getattr(c, "model_hash", False)])
    majority_hash, _count = hash_counts.most_common(1)[0]

    filtered_commits = [(c, n) for c, n in most_updated_commits if getattr(c, "model_hash", False) == majority_hash]
    if not filtered_commits:
        return False, []

    # 3) prepare download_meta for each entry (uid, ip, port, model_hash, model_version)
    download_meta = []
    for commit, neuron in filtered_commits:
        # Only include entries with reachable metadata
        try:
            download_meta.append(
                {
                    "uid": neuron.uid,
                    "ip": neuron.axon_info.ip,
                    "port": neuron.axon_info.port,
                    "model_hash": commit.model_hash,
                    "global_ver": commit.global_ver,
                    "target_hotkey_ss58": neuron.hotkey,
                }
            )

        except Exception:
            logger.info("Cannot append commit", commit=commit)

    # should_download if there is at least one newer entry agreeing on a majority hash
    should_download = len(download_meta) > 0

    return should_download, download_meta

scan_chain_for_new_model(
    ModelMeta(global_ver=480, inner_opt=20), miner_config, subtensor
)

2025-12-23 00:48:13 [info     ] Scan chain: Max model version on chain [__main__] max_model_version_on_chain=ModelMeta(global_ver=493, inner_opt=0, path=None, role=None, model_hash=None)
2025-12-23 00:48:13 [info     ] Scan chain: Local model version [__main__] current_model_version=ModelMeta(global_ver=480, inner_opt=20, path=None, role=None, model_hash=None)
2025-12-23 00:48:13 [info     ] Scan chain: Max model meta   [__main__] max_model_meta=ModelMeta(global_ver=493, inner_opt=0, path=None, role=None, model_hash=None)
2025-12-23 00:48:13 [info     ] commits A                    [__main__] positional_args=((ValidatorChainCommit(model_hash='5baaf29d41aec8eed6a2262619a8d5f5f670f03a8fd25a8e', global_ver=493, expert_group=0, miner_seed=39062, block=6096567), NeuronInfoLite(hotkey='5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH', coldkey='5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr', uid=16, netuid=348, active=False, stake=τ0.000000000, stake_dict={'5EXGTzS8r36MrKuYGknxmyqsX4eModU

(True,
 [{'uid': 16,
   'ip': '149.137.225.62',
   'port': 8001,
   'model_hash': '5baaf29d41aec8eed6a2262619a8d5f5f670f03a8fd25a8e',
   'global_ver': 493,
   'target_hotkey_ss58': '5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH'}])

## Commit 